# Imports

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs
!pip install transformers[torch]
!pip install accelerate -U
!pip install spacy
!pip install gensim
!pip install xgboost
!pip install numpy scikit-learn
!python -m spacy download fr_core_news_sm
!pip install sacremoses
!pip install transformers torch sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using c

In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import string
import nltk
import torch
import torch.nn as nn
import torch.optim as optim

from datasets import Dataset,load_metric
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic
from nltk.corpus import wordnet
from textblob import TextBlob
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline, DataCollatorWithPadding, FlaubertTokenizer, FlaubertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from gensim.models import Word2Vec
from gensim.models import FastText
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Download NLTK resources
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw')
nltk.download('wordnet_ic')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.


True

In [ ]:
# Import files
sample_submission = pd.read_csv("https://raw.githubusercontent.com/allobo49/SBB/main/Data/sample_submission.csv")
training_data = pd.read_csv("https://raw.githubusercontent.com/allobo49/SBB/main/Data/training_data.csv")
unlabelled_test_data = pd.read_csv("https://raw.githubusercontent.com/allobo49/SBB/main/Data/unlabelled_test_data.csv")

In [ ]:
# Read sample_submission
sample_submission

,id,difficulty
0,0,A1
1,1,A1
2,2,A1
3,3,A1
4,4,A1
...,...,...
1195,1195,A1
1196,1196,A1
1197,1197,A1
1198,1198,A1


In [ ]:
# Read training_data
training_data

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1
...,...,...,...
4795,4795,"C'est pourquoi, il décida de remplacer les hab...",B2
4796,4796,Il avait une de ces pâleurs splendides qui don...,C1
4797,4797,"Et le premier samedi de chaque mois, venez ren...",A2
4798,4798,Les coûts liés à la journalisation n'étant pas...,C2


In [ ]:
# Read unlabelled_test_data
unlabelled_test_data

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."
...,...,...
1195,1195,C'est un phénomène qui trouve une accélération...
1196,1196,Je vais parler au serveur et voir si on peut d...
1197,1197,Il n'était pas comme tant de gens qui par pare...
1198,1198,Ils deviennent dangereux pour notre économie.


# Data Augmentation and Preparation

In [ ]:
# Function to count words in each sentence excluding stop words
def count_words(sentence):
    # Tokenize the sentence
    words = sentence.split()
    # Return the count of non-stop words
    return len(words)

In [ ]:
# Function to add a dot at the end of the sentence if it doesn't end with a punctuation mark
def add_dot(sentence):
    if sentence[-1] not in string.punctuation:
        return sentence + '.'
    else:
        return sentence

In [ ]:
# Function to calculate average word length in a sentence
def avg_word_length(sentence):
    words = sentence.split()
    word_lengths = [len(word) for word in words]
    if len(word_lengths) == 0:
        return 0
    return sum(word_lengths) / len(word_lengths)

In [ ]:
# Function to count punctuation marks in a sentence
def count_punctuation(sentence):
    punctuation_count = sum([1 for char in sentence if char in string.punctuation])
    return punctuation_count

In [ ]:
# Function to count stop words in each sentence
def stopword_proportion(sentence):
    words = sentence.split()
    total_words = len(words)
    stop_words_in_sentence = [word for word in words if word.lower() in stop_words]
    num_stopwords = len(stop_words_in_sentence)
    if total_words == 0:
        return 0
    return num_stopwords / total_words

In [ ]:
# Function to get POS tags for each word in a sentence
def get_pos_tags(sentence):
    words = sentence.split()
    pos_tags = pos_tag(words)
    return pos_tags

In [ ]:
# Function to calculate readability score of a sentence (Flesch-Kincaid)
def flesch_kincaid_readability(sentence):
    words = sentence.split()
    num_words = len(words)
    num_sentences = 1  # assuming each input is a single sentence
    num_syllables = sum([syllable_count(word) for word in words])
    readability = 206.835 - 1.015 * (num_words / num_sentences) - 84.6 * (num_syllables / num_words)
    return readability

# Function to count syllables in a word for readability score calculation
def syllable_count(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith('e'):
        count -= 1
    if count == 0:
        count += 1
    return count

In [ ]:
# Load the French language model
import spacy
nlp = spacy.load('fr_core_news_sm')

# Function to analyze parts of speech in a sentence
def analyze_pos(sentence):
    doc = nlp(sentence)
    pos_counts = {}
    for token in doc:
        if token.pos_ in pos_counts:
            pos_counts[token.pos_] += 1
        else:
            pos_counts[token.pos_] = 1
    return pos_counts

## TRAINING DATA
# Apply the function to each sentence
training_data['pos_counts'] = training_data['sentence'].apply(analyze_pos)

# Create columns for each part of speech count
pos_set = set()
training_data['pos_counts'].apply(lambda counts: pos_set.update(counts.keys()))
for pos in pos_set:
    training_data[pos] = training_data['pos_counts'].apply(lambda counts: counts.get(pos, 0))

# Drop the temporary 'pos_counts' column
training_data.drop(columns=['pos_counts'], inplace=True)


## DATA PREDICTIONS
# Apply the function to each sentence
unlabelled_test_data['pos_counts'] = unlabelled_test_data['sentence'].apply(analyze_pos)

# Create columns for each part of speech count
pos_set = set()
unlabelled_test_data['pos_counts'].apply(lambda counts: pos_set.update(counts.keys()))
for pos in pos_set:
    unlabelled_test_data[pos] = unlabelled_test_data['pos_counts'].apply(lambda counts: counts.get(pos, 0))


# Drop the temporary 'pos_counts' column
unlabelled_test_data.drop(columns=['pos_counts'], inplace=True)

In [ ]:
# Apply the functions to the DataFrame
training_data['count_words'] = training_data['sentence'].apply(count_words)
training_data['sentence'] = training_data['sentence'].apply(add_dot)
training_data['avg_word_length'] = training_data['sentence'].apply(avg_word_length)
training_data['punctuation_count'] = training_data['sentence'].apply(count_punctuation)
training_data['stopword_proportion'] = training_data['sentence'].apply(stopword_proportion)
training_data['flesch_kincaid_readability'] = training_data['sentence'].apply(flesch_kincaid_readability)

In [ ]:
training_data

,id,sentence,difficulty,DET,INTJ,PUNCT,PRON,ADP,PROPN,ADV,...,CCONJ,X,VERB,NUM,ADJ,count_words,avg_word_length,punctuation_count,stopword_proportion,flesch_kincaid_readability
0,0,Les coûts kilométriques réels peuvent diverger...,C1,3,0,5,0,13,0,2,...,1,0,4,0,3,38,5.736842,8,0.394737,14.649211
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1,3,0,2,2,0,1,2,...,1,0,2,0,0,12,4.250000,4,0.500000,103.005000
2,2,Le test de niveau en français est sur le site ...,A1,3,0,1,0,4,0,0,...,0,0,1,0,0,13,4.153846,2,0.538462,76.501538
3,3,Est-ce que ton mari est aussi de Boston?,A1,0,0,1,1,1,1,1,...,0,0,1,0,0,8,4.125000,2,0.500000,82.390000
4,4,"Dans les écoles de commerce, dans les couloirs...",B1,4,0,5,2,11,0,1,...,1,0,4,2,2,34,5.176471,9,0.411765,40.448529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4795,4795,"C'est pourquoi, il décida de remplacer les hab...",B2,4,0,3,4,4,0,2,...,0,0,7,0,1,26,5.384615,4,0.384615,30.768077
4796,4796,Il avait une de ces pâleurs splendides qui don...,C1,3,0,1,3,5,0,0,...,0,0,3,0,1,21,4.666667,1,0.523810,60.634286
4797,4797,"Et le premier samedi de chaque mois, venez ren...",A2,3,0,2,0,2,0,0,...,1,0,2,0,1,14,4.785714,2,0.357143,59.682143
4798,4798,Les coûts liés à la journalisation n'étant pas...,C2,7,0,2,0,3,0,4,...,2,0,2,0,4,32,6.093750,5,0.406250,26.305000


In [ ]:
features = ['count_words', 'avg_word_length', 'punctuation_count', 'stopword_proportion',
                   'flesch_kincaid_readability', 'ADV', 'SCONJ', 'NOUN', 'INTJ', 'DET', 'ADP',
                   'PRON', 'X', 'ADJ', 'AUX', 'PROPN', 'CCONJ', 'VERB', 'NUM']

# Logistic Regression

## Only Sentence

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data['sentence']
y = training_data['difficulty']

random_seed = 123
np.random.seed(random_seed)

# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer()

# Create a pipeline that includes TF-IDF vectorization followed by logistic regression
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000, random_state=random_seed))
])

# Define the parameter grid to search
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'tfidf__use_idf': [True, False],
    'classifier__C': [0.01, 0.1, 1, 10],
    'classifier__penalty': ['l2', 'l1', 'none'],
    'classifier__solver': ['liblinear', 'lbfgs']  # liblinear works with l1 penalty, lbfgs with l2 and none
}

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Setup GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and model
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_LR_sentence = grid_search.best_estimator_
y_pred = best_model_LR_sentence.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

## Additional Features

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data[features + ['sentence']]
y = training_data['difficulty']

# Set random seed for reproducibility
random_seed = 123
np.random.seed(random_seed)

# Define the column transformer to account for the sentences and additional features
column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'sentence'),
        ('scaler', StandardScaler(), features)
    ], remainder='passthrough')

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', LogisticRegression(max_iter=1000, random_state=random_seed))
])

# Define the parameter grid
param_grid = {
    'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2)],
    'preprocessor__tfidf__use_idf': [True, False],
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l2', 'none']
}

# Setup GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2, scoring='accuracy')

In [ ]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Perform grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_LR_features = grid_search.best_estimator_
y_pred = best_model_LR_features.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

In [ ]:
# Print detailed classification metrics
print(classification_report(y_test, y_pred))

## Additional features (no POS)

In [36]:
features = ['count_words', 'avg_word_length', 'punctuation_count', 'stopword_proportion',
                   'flesch_kincaid_readability']

In [37]:
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data[features + ['sentence']]
y = training_data['difficulty']

# Set random seed for reproducibility
random_seed = 123
np.random.seed(random_seed)

# Define the column transformer to account for the sentences and additional features
column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'sentence'),
        ('scaler', StandardScaler(), features)
    ], remainder='passthrough')

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', LogisticRegression(max_iter=1000, random_state=random_seed))
])

# Define the parameter grid
param_grid = {
    'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2)],
    'preprocessor__tfidf__use_idf': [True, False],
    'classifier__C': [0.1, 1, 10],
    'classifier__penalty': ['l2', 'none']
}

# Setup GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2, scoring='accuracy')

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Perform grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_LR_features = grid_search.best_estimator_
y_pred = best_model_LR_features.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

# Print detailed classification metrics
print(classification_report(y_test, y_pred))

# Save the best model
joblib.dump(best_model_LR_features, 'best_model_LR_features.joblib')
print("Model saved as 'best_model_LR_features.joblib'")

import joblib
from google.colab import files

# Save the model to Colab's local file system
model_filename = 'best_model_LR_features.joblib'
joblib.dump(best_model_LR_features, model_filename)

# Trigger a browser download
files.download(model_filename)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] END classifier__C=0.1, classifier__penalty=l2, preprocessor__tfidf__ngram_range=(1, 1), preprocessor__tfidf__use_idf=True; total time=   1.0s
[CV] END classifier__C=0.1, classifier__penalty=l2, preprocessor__tfidf__ngram_range=(1, 1), preprocessor__tfidf__use_idf=True; total time=   1.6s
[CV] END classifier__C=0.1, classifier__penalty=l2, preprocessor__tfidf__ngram_range=(1, 1), preprocessor__tfidf__use_idf=True; total time=   2.0s
[CV] END classifier__C=0.1, classifier__penalty=l2, preprocessor__tfidf__ngram_range=(1, 1), preprocessor__tfidf__use_idf=False; total time=   1.9s
[CV] END classifier__C=0.1, classifier__penalty=l2, preprocessor__tfidf__ngram_range=(1, 1), preprocessor__tfidf__use_idf=False; total time=   2.2s
[CV] END classifier__C=0.1, classifier__penalty=l2, preprocessor__tfidf__ngram_range=(1, 1), preprocessor__tfidf__use_idf=False; total time=   1.6s
[CV] END classifier__C=0.1, classifier__penalty=l2, pr

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END classifier__C=0.1, classifier__penalty=none, preprocessor__tfidf__ngram_range=(1, 1), preprocessor__tfidf__use_idf=True; total time=   2.4s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END classifier__C=0.1, classifier__penalty=none, preprocessor__tfidf__ngram_range=(1, 1), preprocessor__tfidf__use_idf=True; total time=   3.8s


...

[CV] END classifier__C=10, classifier__penalty=none, preprocessor__tfidf__ngram_range=(1, 2), preprocessor__tfidf__use_idf=False; total time=   9.5s


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


[CV] END classifier__C=10, classifier__penalty=none, preprocessor__tfidf__ngram_range=(1, 2), preprocessor__tfidf__use_idf=False; total time=   7.8s
Best parameters found:  {'classifier__C': 10, 'classifier__penalty': 'l2', 'preprocessor__tfidf__ngram_range': (1, 2), 'preprocessor__tfidf__use_idf': True}
Best cross-validation accuracy:  0.47109375000000003
Test Accuracy with best model:  0.49895833333333334
              precision    recall  f1-score   support

           0       0.56      0.73      0.64       153
           1       0.48      0.44      0.46       156
           2       0.44      0.49      0.46       153
           3       0.48      0.50      0.49       173
           4       0.44      0.31      0.37       166
           5       0.57      0.54      0.56       159

    accuracy                           0.50       960
   macro avg       0.50      0.50      0.49       960
weighted avg       0.49      0.50      0.49       960

Model saved as 'best_model_LR_features.joblib'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Random Forest Classifier

## Only Sentence

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data['sentence']
y = training_data['difficulty']

random_seed = 123
np.random.seed(random_seed)

# Create a pipeline that includes TF-IDF vectorization followed by RandomForestClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', RandomForestClassifier(random_state=random_seed))
])

# Define the parameter grid to search
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': [True, False],
    'classifier__n_estimators': [100, 200, 300],  # Number of trees in the forest
    'classifier__max_depth': [None, 10, 20, 30],  # Maximum number of levels in each decision tree
    'classifier__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'classifier__min_samples_leaf': [1, 2, 4]     # Minimum number of samples required at each leaf node
}

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Setup GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and model
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_RF_sentence = grid_search.best_estimator_
y_pred = best_model_RF_sentence.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

## Additional Features

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data[features + ['sentence']]
y = training_data['difficulty']

random_seed = 123
np.random.seed(random_seed)

# Define the column transformer to account for the sentences and the additional features
column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'sentence'),  # Apply TF-IDF to the 'sentence' column
        ('scaler', StandardScaler(), features)     # Apply scaling to features columns
    ], remainder='passthrough')

# Define the pipeline with Random Forest as the classifier
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', RandomForestClassifier(random_state=random_seed))
])

# Define the parameter grid for Random Forest and TF-IDF
param_grid = {
    'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2)],
    'preprocessor__tfidf__use_idf': [True, False],
    'classifier__n_estimators': [100, 200, 300],  # Number of trees in the forest
    'classifier__max_depth': [None, 10, 20, 30],  # Maximum number of levels in each decision tree
    'classifier__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'classifier__min_samples_leaf': [1, 2, 4]    # Minimum number of samples required at each leaf node
}

In [ ]:
# Setup GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=3, verbose=2, scoring='accuracy')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_RF_features = grid_search.best_estimator_
y_pred = best_model_RF_features.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

# Decision Tree Classifier

## Only Sentence

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data['sentence']
y = training_data['difficulty']

random_seed = 123
np.random.seed(random_seed)

# Create a pipeline that includes TF-IDF vectorization followed by Decision Tree classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', DecisionTreeClassifier(random_state=random_seed))
])

# Define the parameter grid to search for Decision Tree
param_grid = {
    'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2)],
    'preprocessor__tfidf__use_idf': [True, False],
    'classifier__max_depth': [None, 10, 20, 30],  # Maximum number of levels in each decision tree
    'classifier__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'classifier__min_samples_leaf': [1, 2, 4]      # Minimum number of samples required at each leaf node
}

In [ ]:
# Setup GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, scoring='accuracy')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_DT_sentence = grid_search.best_estimator_
y_pred = best_model_DT_sentence.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

## Additional Features

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data[features + ['sentence']]
y = training_data['difficulty']

random_seed = 123
np.random.seed(random_seed)

# Define the column transformer to account for the sentences and the additional features
column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'sentence'),
        ('scaler', StandardScaler(), features)
    ], remainder='passthrough')

# Update the pipeline with a Decision Tree classifier
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', DecisionTreeClassifier(random_state=random_seed))
])

# Update the parameter grid for the Decision Tree
param_grid = {
    'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2)],
    'preprocessor__tfidf__use_idf': [True, False],
    'classifier__max_depth': [None, 10, 20, 30],  # Control over-fitting by limiting the depth of trees
    'classifier__min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'classifier__min_samples_leaf': [1, 2, 4]    # Minimum number of samples required to be at a leaf node
}

In [ ]:
# Setup GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, scoring='accuracy')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_DT_features = grid_search.best_estimator_
y_pred = best_model_DT_features.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

In [ ]:
# Print detailed classification metrics
print(classification_report(y_test, y_pred))

# KNN Classifier

## Only Sentence

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data['sentence']
y = training_data['difficulty']

random_seed = 123
np.random.seed(random_seed)

# Create a pipeline that includes TF-IDF vectorization followed by KNN classifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', KNeighborsClassifier())
])

# Define the parameter grid to search for KNN
param_grid = {
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': [True, False],
    'classifier__n_neighbors': [3, 5, 6, 7],  # Number of neighbors to use
    'classifier__weights': ['uniform', 'distance'],  # Weight function used in prediction
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # Algorithm used to compute the nearest neighbors
}

In [ ]:
# Setup GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, scoring='accuracy')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_KNN_sentence = grid_search.best_estimator_
y_pred = best_model_KNN_sentence.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

## Additional Features

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])

# Define X and y
X = training_data[features + ['sentence']]
y = training_data['difficulty']

random_seed = 123
np.random.seed(random_seed)

# Define the column transformer for handling sentences and additional features
column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'sentence'),
        ('scaler', StandardScaler(), features)
    ], remainder='passthrough')

# Update the pipeline with a KNN classifier
pipeline = Pipeline([
    ('preprocessor', column_transformer),
    ('classifier', KNeighborsClassifier())
])

# Update the parameter grid for KNN
param_grid = {
    'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2)],
    'preprocessor__tfidf__use_idf': [True, False],
    'classifier__n_neighbors': [3, 5, 6, 7],  # Number of neighbors
    'classifier__weights': ['uniform', 'distance'],  # Weighting type
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # Algorithm to compute nearest neighbors
}

In [ ]:
# Setup GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, verbose=2, scoring='accuracy')

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: ", grid_search.best_score_)

# Test the best model on the test data
best_model_KNN_features = grid_search.best_estimator_
y_pred = best_model_KNN_features.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with best model: ", test_accuracy)

In [ ]:
# Print classification report
print(classification_report(y_test, y_pred))

# Neural Networks

## One Sentence

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])
y = training_data['difficulty']

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(training_data['sentence']).toarray()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale X features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled X_train and X_test to torch tensors of type torch.float
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float)

# Convert y_train and y_test to torch tensors of type torch.long
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

In [ ]:
# Define the neural network class
class NeuralNetwork(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(NeuralNetwork, self).__init__()

        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, D_out)
        self.activation = nn.ReLU()

    def forward(self, x):
        y_pred = self.activation(self.linear1(x))
        y_pred = self.linear2(y_pred)
        return y_pred

In [ ]:
# Hyperparameters
iterations = [500, 1000, 1500]
learning_rates = [0.001, 1.049, 12.031]

# Input and output sizes
D_in = X_train_tensor.shape[1]
D_out = len(label_encoder.classes_)

# Accuracy Initialization
best_accuracy = 0
best_model = None

# Training and evaluation of the model
for iteration in iterations:
    for lr in learning_rates:
        model = NeuralNetwork(D_in, 150, D_out)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()

        # Training loop
        for i in range(iteration):
            model.train()
            optimizer.zero_grad()
            y_pred = model(X_train_tensor)
            loss = criterion(y_pred, y_train_tensor)

            # Backward pass
            loss.backward()
            optimizer.step()

        # Evaluation
        with torch.no_grad():
            correct = 0
            total = 0
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs.data, 1)
            total += y_test_tensor.size(0)
            correct += (predicted == y_test_tensor).sum().item()

        # Print accuracy
        accuracy = correct*100 / total
        print(f"Iterations: {iteration}, Learning Rate: {lr}, Accuracy: {accuracy}%")

        # Save the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

# Save the best model
torch.save(best_model.state_dict(), 'best_model.pth')
print("Best model saved with accuracy:", best_accuracy)

In [ ]:
# Convert class labels to string if necessary
class_names = [str(cls) for cls in label_encoder.classes_]

# Ensure the best model is in evaluation mode
best_model.eval()

# Generate predictions
with torch.no_grad():
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

# Convert predictions and true labels to CPU numpy arrays if running on a GPU
predicted = predicted.cpu().numpy()
y_true = y_test_tensor.cpu().numpy()

# Generate the classification report
report = classification_report(y_true, predicted, target_names=class_names)
print(report)

## Additional Features

In [ ]:
# Encode the 'difficulty' label
label_encoder = LabelEncoder()
training_data["difficulty"] = label_encoder.fit_transform(training_data["difficulty"])
y = training_data['difficulty'].values

# Define column transformer accounting for sentences and additional features
column_transformer = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'sentence'),
        ('scaler', StandardScaler(), features)
    ],
    remainder='passthrough'
)

# Prepare X by fitting and transforming the data
X = column_transformer.fit_transform(training_data[features + ['sentence']])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Convert sparse matrix to dense
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Convert arrays to tensors
X_train_tensor = torch.tensor(X_train_dense, dtype=torch.float)
X_test_tensor = torch.tensor(X_test_dense, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Neural network definition
class NeuralNetwork(nn.Module):
    def __init__(self, D_in, H1, D_out):
        super(NeuralNetwork, self).__init__()
        self.linear1 = nn.Linear(D_in, H1)
        self.activation = nn.ReLU()
        self.linear2 = nn.Linear(H1, D_out)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        return x

In [ ]:
# Hyperparameters
iterations = [500, 1000, 1500]
learning_rates = [0.001, 1.049, 12.031]

# Input and output sizes
D_in = X_train_tensor.shape[1]  # Correct number of input features
D_out = len(label_encoder.classes_)  # Number of classes

# Accuracy Initialization
best_accuracy = 0
best_model = None

# Training and evaluation of the model
for iteration in iterations:
    for lr in learning_rates:
        model = NeuralNetwork(D_in, 150, D_out)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()

        # Training loop
        for i in range(iteration):
            model.train()
            optimizer.zero_grad()
            y_pred = model(X_train_tensor)
            loss = criterion(y_pred, y_train_tensor)
            loss.backward()
            optimizer.step()

        # Evaluation
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            outputs = model(X_test_tensor)
            _, predicted = torch.max(outputs.data, 1)
            total += y_test_tensor.size(0)
            correct += (predicted == y_test_tensor).sum().item()
            accuracy = correct * 100 / total

        print(f"Iterations: {iteration}, Learning Rate: {lr}, Accuracy: {accuracy}%")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = model

# Save the best model
torch.save(best_model.state_dict(), 'best_model.pth')
print("Best model saved with accuracy:", best_accuracy)

Iterations: 500, Learning Rate: 0.001, Accuracy: 18.854166666666668%
Iterations: 500, Learning Rate: 1.049, Accuracy: 43.125%
Iterations: 500, Learning Rate: 12.031, Accuracy: 15.833333333333334%
Iterations: 1000, Learning Rate: 0.001, Accuracy: 30.833333333333332%
Iterations: 1000, Learning Rate: 1.049, Accuracy: 44.270833333333336%
Iterations: 1000, Learning Rate: 12.031, Accuracy: 16.666666666666668%
Iterations: 1500, Learning Rate: 0.001, Accuracy: 28.541666666666668%
Iterations: 1500, Learning Rate: 1.049, Accuracy: 50.104166666666664%
Iterations: 1500, Learning Rate: 12.031, Accuracy: 16.041666666666668%
Best model saved with accuracy: 50.104166666666664


In [ ]:
# Convert class labels to string if necessary
class_names = [str(cls) for cls in label_encoder.classes_]

# Ensure the best model is in evaluation mode
best_model.eval()

# Generate predictions
with torch.no_grad():
    outputs = best_model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)

# Convert predictions and true labels to CPU numpy arrays if running on a GPU
predicted = predicted.cpu().numpy()
y_true = y_test_tensor.cpu().numpy()

# Generate the classification report
report = classification_report(y_true, predicted, target_names=class_names)
print(report)

              precision    recall  f1-score   support

           0       0.75      0.36      0.49       153
           1       0.43      0.78      0.56       156
           2       0.56      0.45      0.50       153
           3       0.50      0.49      0.49       173
           4       0.42      0.58      0.49       166
           5       0.64      0.34      0.44       159

    accuracy                           0.50       960
   macro avg       0.55      0.50      0.50       960
weighted avg       0.55      0.50      0.49       960

